<a href="https://colab.research.google.com/github/Kufzuk2/Comp_math/blob/main/Task2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np

def jacobi(A, b, tol=1e-16, max_iterations=1000):

    n = len(A)
    x = np.zeros_like(b, dtype=np.double)  # Начальное приближение нулевым вектором
    x_new = np.zeros_like(b, dtype=np.double) # Вектор для новых значений

    for iteration in range(max_iterations):
        print(x)
        for i in range(n):
            # Сумма всех остальных компонент
            s1 = sum(A[i, j] * x_new[j] for j in range(n) if i != j)
            # Обновление i-ой компоненты
            x_new[i] = (b[i] - s1) / A[i, i]

        # Проверка на сходимость
        if np.linalg.norm(x_new - x) < tol:
            print(f"Iter :{iteration}")
            return x_new

        x = x_new.copy()

    print("Not calculating")

# Пример использования
if 1:
  def create_matrix_and_vector(n):
    A = np.ones((n, n))  # Матрица из единиц
    for i in range(1, n):
        A[i, i] = 10  # Устанавливаем диагональные элементы
        if i > 1:
            A[i, i - 1] = 0  # Устанавливаем нули в первой части
        if i > 2:
            A[i, i - 2] = 0  # Устанавливаем нули в первой части
    A[n - 2, n - 3:n] = [1, 10, 1]  # Предпоследняя строка
    A[n - 1, -2:] = [1, 11]  # Последняя строка

    b = np.arange(n, 0, -1)  # Вектор b: 100, 99, ..., 2, 1

    return A, b
    x = jacobi(A, b)
    print("Answer:", x)


In [4]:
import numpy as np

def zeidel(A, b, tol=1e-16, max_iterations=1000):

    n = len(A)
    x = np.zeros_like(b, dtype=np.double)  # Начальное приближение нулевым вектором
    x_new = np.zeros_like(b, dtype=np.double) # Вектор для новых значений

    for iteration in range(max_iterations):
        print(x)
        for i in range(n):
            # Сумма всех остальных компонент
            s1 = sum(A[i, j] * x_new[j] for j in range(i))
            s2 = sum(A[i, j] * x[j] for j in range(i + 1, n))
            s = s1 + s2
            # Обновление i-ой компоненты
            x_new[i] = (b[i] - s) / A[i, i]

        # Проверка на сходимость
        if np.linalg.norm(x_new - x) < tol:
            print(f"Iter :{iteration}")
            return x_new

        x = x_new.copy()

    print("Not calculating")

# Пример использования
if 1:
  def create_matrix_and_vector(n):
    A = np.ones((n, n))  # Матрица из единиц
    for i in range(1, n):
        A[i, i] = 10  # Устанавливаем диагональные элементы
        if i > 1:
            A[i, i - 1] = 0  # Устанавливаем нули в первой части
        if i > 2:
            A[i, i - 2] = 0  # Устанавливаем нули в первой части
    A[n - 2, n - 3:n] = [1, 10, 1]  # Предпоследняя строка
    A[n - 1, -2:] = [1, 11]  # Последняя строка

    b = np.arange(n, 0, -1)  # Вектор b: 100, 99, ..., 2, 1

    return A, b

    x = zeidel(A, b)
    print("Answer:", x)



In [6]:
import numpy as np

def gauss_elimination_with_partial_pivoting(A, b):
    n = len(b)
    Ab = np.hstack([A.astype(float), b.reshape(-1, 1).astype(float)])

    for i in range(n):
        # Поиск индекса максимального элемента в текущем столбце
        max_index = np.argmax(np.abs(Ab[i:n, i])) + i

        # Обмен строк, если необходимо
        if max_index != i:
            Ab[[i, max_index]] = Ab[[max_index, i]]

        # Прямой ход
        for j in range(i + 1, n):
            factor = Ab[j, i] / Ab[i, i]
            Ab[j, :] -= factor * Ab[i, :]

    # Обратный ход
    x = np.zeros(n, dtype=float)
    for i in range(n - 1, -1, -1):
        x[i] = (Ab[i, -1] - np.dot(Ab[i, i + 1:n], x[i + 1:n])) / Ab[i, i]

    return x

def create_matrix_and_vector(n):
    A = np.ones((n, n))  # Матрица из единиц
    for i in range(1, n):
        A[i, i] = 10  # Устанавливаем диагональные элементы
        if i > 1:
            A[i, i - 1] = 0  # Устанавливаем нули в первой части
        if i > 2:
            A[i, i - 2] = 0  # Устанавливаем нули в первой части
    A[n - 2, n - 3:n] = [1, 10, 1]  # Предпоследняя строка
    A[n - 1, -2:] = [1, 11]  # Последняя строка

    b = np.arange(n, 0, -1)  # Вектор b: 100, 99, ..., 2, 1

    return A, b

    x = gauss_elimination_with_partial_pivoting(A, b)
    print("Ответ:", x)

In [7]:
import numpy as np

def sor(A, b, omega, tol=1e-10, max_iterations=1000):
    n = len(b)
    x = np.zeros(n)  # Начальное приближение

    for iteration in range(max_iterations):
        x_old = np.copy(x)

        for i in range(n):
            # Вычисляем сумму
            sigma = 0
            for j in range(n):
                if j != i:
                    sigma += A[i, j] * x[j]

            # Обновление x[i] с использованием верхней релаксации
            x[i] = (1 - omega) * x_old[i] + (omega / A[i, i]) * (b[i] - sigma)

        # Проверка на сходимость
        if np.linalg.norm(x - x_old, ord=np.inf) < tol:
            print(f"Сошелся за {iteration+1} итераций.")
            return x

    print("Не сошелся за максимальное количество итераций.")
    return x
def create_matrix_and_vector(n):
    A = np.ones((n, n))  # Матрица из единиц
    for i in range(1, n):
        A[i, i] = 10  # Устанавливаем диагональные элементы
        if i > 1:
            A[i, i - 1] = 0  # Устанавливаем нули в первой части
        if i > 2:
            A[i, i - 2] = 0  # Устанавливаем нули в первой части
    A[n - 2, n - 3:n] = [1, 10, 1]  # Предпоследняя строка
    A[n - 1, -2:] = [1, 11]  # Последняя строка

    b = np.arange(n, 0, -1)  # Вектор b: 100, 99, ..., 2, 1

    return A, b

# Пример использования
if __name__ == "__main__":
    n = 100
    A, b = create_matrix_and_vector(n)

    omega = 1.25  # Коэффициент релаксации
    x = sor(A, b, omega)
    print("Ответ:", x)

Сошелся за 265 итераций.
Ответ: [ 7.96793359e+02 -1.11111111e-01 -2.34567901e-01 -3.71742112e-01
 -5.11812224e-01 -6.53728260e-01 -7.96171165e-01 -9.38877714e-01
 -1.08167210e+00 -1.22450553e+00 -1.36735307e+00 -1.51020651e+00
 -1.65306218e+00 -1.79591874e+00 -1.93877566e+00 -2.08163271e+00
 -2.22448982e+00 -2.36734695e+00 -2.51020409e+00 -2.65306123e+00
 -2.79591837e+00 -2.93877551e+00 -3.08163265e+00 -3.22448980e+00
 -3.36734694e+00 -3.51020408e+00 -3.65306122e+00 -3.79591837e+00
 -3.93877551e+00 -4.08163265e+00 -4.22448980e+00 -4.36734694e+00
 -4.51020408e+00 -4.65306122e+00 -4.79591837e+00 -4.93877551e+00
 -5.08163265e+00 -5.22448980e+00 -5.36734694e+00 -5.51020408e+00
 -5.65306122e+00 -5.79591837e+00 -5.93877551e+00 -6.08163265e+00
 -6.22448980e+00 -6.36734694e+00 -6.51020408e+00 -6.65306122e+00
 -6.79591837e+00 -6.93877551e+00 -7.08163265e+00 -7.22448980e+00
 -7.36734694e+00 -7.51020408e+00 -7.65306122e+00 -7.79591837e+00
 -7.93877551e+00 -8.08163265e+00 -8.22448980e+00 -8.367346

In [8]:
import numpy as np

def lu_decomposition(A):
    n = len(A)
    L = np.zeros((n, n), dtype=float)
    U = np.zeros((n, n), dtype=float)

    for i in range(n):
        # Верхняя треугольная матрица U
        for j in range(i, n):
            U[i, j] = A[i, j] - np.dot(L[i, :i], U[:i, j])

        # Нижняя треугольная матрица L
        for j in range(i, n):
            if i == j:
                L[i, i] = 1  # Главная диагональ L равна 1
            else:
                L[j, i] = (A[j, i] - np.dot(L[j, :i], U[:i, i])) / U[i, i]

    return L, U

def forward_substitution(L, b):
    n = len(b)
    y = np.zeros(n, dtype=float)

    for i in range(n):
        y[i] = b[i] - np.dot(L[i, :i], y[:i])

    return y

def backward_substitution(U, y):
    n = len(y)
    x = np.zeros(n, dtype=float)

    for i in range(n - 1, -1, -1):
        x[i] = (y[i] - np.dot(U[i, i + 1:], x[i + 1:])) / U[i, i]

    return x

def lu_solver(A, b):
    L, U = lu_decomposition(A)
    y = forward_substitution(L, b)
    x = backward_substitution(U, y)
    return x
def create_matrix_and_vector(n):
    A = np.ones((n, n))  # Матрица из единиц
    for i in range(1, n):
        A[i, i] = 10  # Устанавливаем диагональные элементы
        if i > 1:
            A[i, i - 1] = 0  # Устанавливаем нули в первой части
        if i > 2:
            A[i, i - 2] = 0  # Устанавливаем нули в первой части
    A[n - 2, n - 3:n] = [1, 10, 1]  # Предпоследняя строка
    A[n - 1, -2:] = [1, 11]  # Последняя строка

    b = np.arange(n, 0, -1)  # Вектор b: 100, 99, ..., 2, 1

    return A, b

# Пример использования
if __name__ == "__main__":
    n = 100
    A, b = create_matrix_and_vector(n)

    x = lu_solver(A, b)
    print("Ответ:", x)

Ответ: [ 7.96793359e+02 -1.11111111e-01 -2.34567901e-01 -3.71742112e-01
 -5.11812224e-01 -6.53728260e-01 -7.96171165e-01 -9.38877714e-01
 -1.08167210e+00 -1.22450553e+00 -1.36735307e+00 -1.51020651e+00
 -1.65306218e+00 -1.79591874e+00 -1.93877566e+00 -2.08163271e+00
 -2.22448982e+00 -2.36734695e+00 -2.51020409e+00 -2.65306123e+00
 -2.79591837e+00 -2.93877551e+00 -3.08163265e+00 -3.22448980e+00
 -3.36734694e+00 -3.51020408e+00 -3.65306122e+00 -3.79591837e+00
 -3.93877551e+00 -4.08163265e+00 -4.22448980e+00 -4.36734694e+00
 -4.51020408e+00 -4.65306122e+00 -4.79591837e+00 -4.93877551e+00
 -5.08163265e+00 -5.22448980e+00 -5.36734694e+00 -5.51020408e+00
 -5.65306122e+00 -5.79591837e+00 -5.93877551e+00 -6.08163265e+00
 -6.22448980e+00 -6.36734694e+00 -6.51020408e+00 -6.65306122e+00
 -6.79591837e+00 -6.93877551e+00 -7.08163265e+00 -7.22448980e+00
 -7.36734694e+00 -7.51020408e+00 -7.65306122e+00 -7.79591837e+00
 -7.93877551e+00 -8.08163265e+00 -8.22448980e+00 -8.36734694e+00
 -8.51020408e+00 -

In [9]:
import numpy as np

def gradient_descent(A, b, x0, tol=1e-6, max_iter=1000):
    x = x0
    r = A @ x - b  # начальный градиент

    for i in range(max_iter):
        # градиент
        Ar = A @ r
        alpha = (r @ r) / (Ar @ Ar)  # шаг наискорейшего спуска

        x = x - alpha * r

        # новый остаток
        r = A @ x - b

        # Проверяем условие выхода
        if np.linalg.norm(r) < tol:
            return x

    print("timeout")
    return x

# Пример использования:
def create_matrix_and_vector(n):
    A = np.ones((n, n))  # Матрица из единиц
    for i in range(1, n):
        A[i, i] = 10  # Устанавливаем диагональные элементы
        if i > 1:
            A[i, i - 1] = 0  # Устанавливаем нули в первой части
        if i > 2:
            A[i, i - 2] = 0  # Устанавливаем нули в первой части
    A[n - 2, n - 3:n] = [1, 10, 1]  # Предпоследняя строка
    A[n - 1, -2:] = [1, 11]  # Последняя строка

    b = np.arange(n, 0, -1)  # Вектор b: 100, 99, ..., 2, 1

    return A, b
x0 = np.zeros_like(b)

solution = gradient_descent(A, b, x0)
print("Solution:", solution)


timeout
Solution: [ 2.79152325e+02  3.73012310e+00  4.03356914e+00  4.37063189e+00
  4.33130639e+00  4.25674648e+00  4.13976254e+00  4.01352546e+00
  3.87418535e+00  3.72174575e+00  3.56083147e+00  3.40247321e+00
  3.25656172e+00  3.12501705e+00  3.00093626e+00  2.87411977e+00
  2.73776204e+00  2.59158245e+00  2.44016476e+00  2.28901707e+00
  2.14146452e+00  1.99787215e+00  1.85671636e+00  1.71613791e+00
  1.57494671e+00  1.43282030e+00  1.28999530e+00  1.14686338e+00
  1.00371688e+00  8.60681757e-01  7.17762867e-01  5.74915307e-01
  4.32093952e-01  2.89272293e-01  1.46441366e-01  3.60191506e-03
 -1.39242399e-01 -2.82088483e-01 -4.24934689e-01 -5.67780486e-01
 -7.10625887e-01 -8.53471058e-01 -9.96316137e-01 -1.13916120e+00
 -1.28200627e+00 -1.42485134e+00 -1.56769643e+00 -1.71054152e+00
 -1.85338661e+00 -1.99623170e+00 -2.13907680e+00 -2.28192189e+00
 -2.42476698e+00 -2.56761207e+00 -2.71045716e+00 -2.85330225e+00
 -2.99614734e+00 -3.13899243e+00 -3.28183752e+00 -3.42468261e+00
 -3.567